# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [3]:
pip install dice-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 28.8 MB/s 


In [5]:

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import dice_ml
from dice_ml import Dice
from dice_ml.utils import helpers  # helper functions

In [6]:
%load_ext autoreload
%autoreload 2

## Preliminaries: Loading the data and ML model

In [7]:
dataset = helpers.load_adult_income_dataset().sample(5000)  # downsampling to reduce ML model fitting time
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [8]:
target = dataset["income"]

# Split data into train and test
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [9]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [10]:
exp = Dice(d, m, method="random")
query_instance = x_train[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,
                                  desired_class="opposite",
                                  permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.35it/s]

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,56,Private,School,Married,Professional,White,Male,40,1



Diverse Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,45.0,-,-,-,Other/Unknown,-,-,-,0
1,-,-,-,Single,-,Other,-,-,0
2,-,-,HS-grad,-,-,-,-,22.0,0
3,24.0,-,-,Single,-,-,-,-,0
4,-,Government,-,-,Sales,-,-,-,0
5,-,-,-,Single,-,-,-,32.0,0
6,-,-,-,Single,Sales,-,-,-,0
7,-,-,-,-,White-Collar,-,-,8.0,0
8,40.0,-,-,-,Blue-Collar,-,-,-,0
9,-,-,Assoc,-,Sales,-,-,-,0


These can now be used to calculate the feature importance scores. 

In [11]:
imp = exp.local_feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'occupation': 0.6, 'marital_status': 0.4, 'age': 0.3, 'hours_per_week': 0.3, 'education': 0.2, 'workclass': 0.1, 'race': 0.1, 'gender': 0.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [12]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

100%|██████████| 1/1 [00:00<00:00,  5.31it/s]


[{'age': 0.5, 'marital_status': 0.4, 'occupation': 0.3, 'hours_per_week': 0.3, 'education': 0.2, 'workclass': 0.1, 'race': 0.1, 'gender': 0.1}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [13]:
cobj = exp.global_feature_importance(x_train[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

100%|██████████| 10/10 [00:01<00:00,  6.01it/s]

{'age': 0.55, 'marital_status': 0.36, 'hours_per_week': 0.3, 'education': 0.29, 'occupation': 0.19, 'gender': 0.17, 'race': 0.13, 'workclass': 0.11}


## Convert the counterfactual output to json

In [14]:
json_str = cobj.to_json()
print(json_str)

{"test_data": [[[42, "Self-Employed", "Prof-school", "Married", "Professional", "White", "Male", 50, 1]], [[56, "Private", "School", "Married", "Professional", "White", "Male", 40, 1]], [[39, "Private", "Some-college", "Married", "White-Collar", "White", "Male", 48, 0]], [[31, "Private", "Some-college", "Married", "Service", "Other", "Male", 40, 0]], [[71, "Self-Employed", "Some-college", "Married", "White-Collar", "White", "Male", 50, 1]], [[65, "Private", "HS-grad", "Widowed", "Sales", "White", "Female", 40, 0]], [[67, "Government", "HS-grad", "Widowed", "Service", "White", "Male", 40, 0]], [[25, "Other/Unknown", "School", "Single", "Other/Unknown", "White", "Male", 40, 0]], [[51, "Government", "Some-college", "Married", "Sales", "White", "Male", 40, 1]], [[26, "Private", "Bachelors", "Single", "White-Collar", "Other", "Female", 32, 0]]], "cfs_list": [[[42, "Self-Employed", "HS-grad", "Married", "Professional", "White", "Male", 10.0, 0], [65.0, "Self-Employed", "Prof-school", "Marrie

## Convert the json output to a counterfactual object

In [15]:
imp_r = imp.from_json(json_str)
print([o.visualize_as_dataframe(show_only_changes=True) for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,42,Self-Employed,Prof-school,Married,Professional,White,Male,50,1



Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,HS-grad,-,-,-,-,10.0,0
1,65.0,-,-,-,-,-,Female,-,0
2,-,-,HS-grad,-,Service,-,-,-,0
3,-,Other/Unknown,HS-grad,-,-,-,-,-,0
4,22.0,-,-,-,Blue-Collar,-,-,-,0
5,17.0,Private,-,-,-,-,-,-,0
6,-,-,-,Separated,-,-,Female,-,0
7,68.0,Private,-,-,-,-,-,-,0
8,85.0,-,-,-,-,-,-,-,0
9,-,-,-,Widowed,-,-,Female,-,0


Query instance (original outcome : 1)


...

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,26,Private,Bachelors,Single,White-Collar,Other,Female,32,0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,81.0,-,-,Married,-,-,-,90.0,1
1,65.0,-,-,Married,-,-,-,-,1
2,49.0,-,-,Married,-,-,-,-,1
3,81.0,-,-,Married,-,-,-,-,1
4,-,-,-,Married,-,White,-,72.0,1
5,47.0,-,-,Married,-,-,-,-,1
6,79.0,-,-,Married,-,-,-,-,1
7,-,-,-,Married,-,-,-,64.0,1
8,52.0,-,-,Married,-,-,-,-,1
9,58.0,-,-,Married,-,White,-,-,1


[None, None, None, None, None, None, None, None, None, None]
[{'age': 0.5, 'workclass': 0.3, 'education': 0.3, 'gender': 0.3, 'marital_status': 0.2, 'occupation': 0.2, 'hours_per_week': 0.1, 'race': 0.0}, {'age': 0.5, 'marital_status': 0.4, 'occupation': 0.3, 'gender': 0.3, 'workclass': 0.2, 'race': 0.2, 'hours_per_week': 0.1, 'education': 0.0}, {'age': 0.6, 'hours_per_week': 0.4, 'occupation': 0.3, 'race': 0.2, 'gender': 0.2, 'education': 0.1, 'workclass': 0.0, 'marital_status': 0.0}, {'age': 0.9, 'race': 0.2, 'education': 0.1, 'hours_per_week': 0.1, 'workclass': 0.0, 'marital_status': 0.0, 'occupation': 0.0, 'gender': 0.0}, {'hours_per_week': 0.4, 'race': 0.3, 'age': 0.2, 'workclass': 0.2, 'education': 0.2, 'marital_status': 0.2, 'occupation': 0.2, 'gender': 0.2}, {'education': 0.6, 'marital_status': 0.5, 'hours_per_week': 0.5, 'age': 0.4, 'workclass': 0.0, 'occupation': 0.0, 'race': 0.0, 'gender': 0.0}, {'marital_status': 0.8, 'education': 0.4, 'age': 0.3, 'hours_per_week': 0.2, 'oc